In [ ]:
# load ztf data
from pathlib import Path
from datetime import datetime
import pandas as pd
import lasair
import numpy as np
from utils import processLasairData

project_root = Path.cwd().parent
folder_name = input("Enter name for run folder (leave blank for timestamp): ").strip()
if not folder_name:
    start_time = datetime.now()
    folder_name = start_time.strftime("%Y-%m-%d_%H-%M-%S")
run_folder = project_root / "runs" / folder_name
run_folder.mkdir(parents=True, exist_ok=True)

ztf_data = project_root / "ztf_cleansed.csv"

ztf_df = pd.read_csv(ztf_data)

print(ztf_df.head())

Enter name for run folder (leave blank to use timestamp): full_test_1
          ZTFID      IAUID           RA          Dec     peakt peakfilt  \
0  ZTF17aabtvsy  SN2022yei  10:35:32.09  +37:38:59.0  59870.49        r   
1  ZTF17aabvong  SN2024xxq  02:05:07.68  +11:14:55.1  60606.25        g   
2  ZTF17aacldgo  SN2022zxv  03:09:24.36  -04:53:39.2  59897.25        g   
3  ZTF17aadlxmv  SN2020adv  08:29:47.59  +33:54:22.8  58879.19        g   
4  ZTF18aaaonon  SN2022jjs  10:19:05.51  +14:24:16.6  59703.27        g   

   peakmag  peakabs duration    rise      fade   type  redshift          b  \
0  18.0303   -19.41  >34.229   >6.01    28.219  SN Ia   0.06922  59.641962   
1  16.8039   -19.57   23.222   8.464    14.758  SN Ia   0.03400 -47.664064   
2  18.7979   -18.91    >1077   >3.85  >1073.15  SN Ia   0.07200 -50.332472   
3  17.9475   -19.34   25.146  10.951    14.195  SN Ia   0.06200  34.174702   
4  18.5663   -19.08  >18.631   3.691    >14.94  SN Ia   0.07141  52.363911   

     A_V  

In [ ]:
from typing import Any

lasair_token = 'c4ce6509f05363cfb57aaedffb056ef31573e647'
client = lasair.lasair_client(lasair_token)
print("Lasair client initialized.")

num_total = len(ztf_df)
num_processed = 0
num_success = 0

for idx, row in enumerate[Any](ztf_df.itertuples(index=False)):
    num_processed += 1
    print(f"\nProcessing {num_processed:3d}/{num_total}: {row.ZTFID}...", end='')
    ztf_id = row.ZTFID
    iauid = row.IAUID
    if str(iauid).startswith('SN'):
        iauid = str(iauid)[2:]

    try:
        result = client.object(ztf_id)
    except Exception as e:
        print(f"\nLasair API error for {ztf_id}: {e}")
        continue

    if not result or 'candidates' not in result:
        print(f"\nNo candidates found for {ztf_id}. The object may not exist or API error.")
        continue
    if "error" in result:
        print(f"\nAPI returned error for {ztf_id}: {result.get('error')}")
        continue

    candidates = result['candidates']
    if not candidates:
        print(f"\nNo light curve data for {ztf_id}")
        continue

    output_rows = []
    for cand in candidates:
        # fid: filter ID (1=g, 2=r)
        fid = cand.get("fid")
        fid_to_letter = {1: "g", 2: "r", 3: "i"}
        filter_letter = fid_to_letter.get(fid, fid) if fid is not None else None

        out_row = {
            'ztf_id': ztf_id,
            'iauid': iauid,
            'MJD': cand.get('mjd', None),
            'filter': filter_letter,
            'unforced_mag': cand.get('magpsf', None),
            'unforced_mag_error': cand.get('sigmapsf', None),
            'unforced_mag_status': 'positive' if cand.get('isdiffpos', 't') == 't' else 'negative',
            'forced_ujy': cand.get('forcediffimflux', None),
            'forced_ujy_error': cand.get('forcediffimfluxunc', None),
        }
        output_rows.append(out_row)

    # Save per-object light curve CSV in run_folder
    df_obj = pd.DataFrame(output_rows)
    out_path = run_folder / f"{ztf_id}_lightcurve.csv"
    df_obj.to_csv(out_path, index=False)
    print(f" Saved {len(output_rows)} rows to {out_path.name}.", end=' ')
    try:
        processLasairData(df_obj, out_path)
        num_success += 1
        print(f"Done ({num_success}/{num_processed})")
    except Exception as e:
        print(f"Error running processLasairData: {e} ({num_success}/{num_processed})")

print(f"\nAll done! {num_success} of {num_total} objects processed successfully.")

Lasair client initialized.

Processing   1/425: ZTF17aabtvsy... Saved 60 rows to ZTF17aabtvsy_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF17aabtvsy_lightcurve.csv (23 rows)
Done (1/1)

Processing   2/425: ZTF17aabvong... Saved 85 rows to ZTF17aabvong_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF17aabvong_lightcurve.csv (31 rows)
Done (2/2)

Processing   3/425: ZTF17aacldgo... Saved 74 rows to ZTF17aacldgo_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF17aacldgo_lightcurve.csv (17 rows)
Done (3/3)

Processing   4/425: ZTF17aadlxmv... Saved 71 rows to ZTF17aadlxmv_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF17aadlxmv_lightcurve.csv (28 rows)
Done (4/4)

Processing   5/425: ZTF18aaaonon... Saved 105 rows to ZTF18aaaonon_lightcurve.csv. S

Processing  38/425: ZTF18aansqom... Saved 60 rows to ZTF18aansqom_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18aansqom_lightcurve.csv (33 rows)
Done (38/38)

Processing  39/425: ZTF18aansqov... Saved 53 rows to ZTF18aansqov_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18aansqov_lightcurve.csv (21 rows)
Done (39/39)

Processing  40/425: ZTF18aansqun... Saved 13 rows to ZTF18aansqun_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18aansqun_lightcurve.csv (13 rows)
Done (40/40)

Processing  41/425: ZTF18aanxlcr... Saved 35 rows to ZTF18aanxlcr_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18aanxlcr_lightcurve.csv (20 rows)
Done (41/41)

Processing  42/425: ZTF18aanygjk... Saved 111 rows to ZTF18aanygjk_lightcurve.csv. Successfully cleaned 

Processing  75/425: ZTF18aatwqed... Saved 77 rows to ZTF18aatwqed_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18aatwqed_lightcurve.csv (30 rows)
Done (75/75)

Processing  76/425: ZTF18aauizcr... Saved 837 rows to ZTF18aauizcr_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18aauizcr_lightcurve.csv (391 rows)
Done (76/76)

Processing  77/425: ZTF18aaunfqq... Saved 18 rows to ZTF18aaunfqq_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18aaunfqq_lightcurve.csv (18 rows)
Done (77/77)

Processing  78/425: ZTF18aauotcv... Saved 58 rows to ZTF18aauotcv_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18aauotcv_lightcurve.csv (26 rows)
Done (78/78)

Processing  79/425: ZTF18aavdwmh... Saved 33 rows to ZTF18aavdwmh_lightcurve.csv. Successfully cleaned

Processing 112/425: ZTF18aaxvpsw... Saved 14 rows to ZTF18aaxvpsw_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18aaxvpsw_lightcurve.csv (14 rows)
Done (112/112)

Processing 113/425: ZTF18aaxwjmp... Saved 35 rows to ZTF18aaxwjmp_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18aaxwjmp_lightcurve.csv (35 rows)
Done (113/113)

Processing 114/425: ZTF18aaybfsd... Saved 103 rows to ZTF18aaybfsd_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18aaybfsd_lightcurve.csv (60 rows)
Done (114/114)

Processing 115/425: ZTF18aaydmkh... Saved 14 rows to ZTF18aaydmkh_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18aaydmkh_lightcurve.csv (14 rows)
Done (115/115)

Processing 116/425: ZTF18aayiahw... Saved 8 rows to ZTF18aayiahw_lightcurve.csv. Successfully c

Processing 149/425: ZTF18abdfwur... Saved 24 rows to ZTF18abdfwur_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18abdfwur_lightcurve.csv (24 rows)
Done (149/149)

Processing 150/425: ZTF18abdfydj... Saved 19 rows to ZTF18abdfydj_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18abdfydj_lightcurve.csv (19 rows)
Done (150/150)

Processing 151/425: ZTF18abdgwxn... Saved 10 rows to ZTF18abdgwxn_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18abdgwxn_lightcurve.csv (10 rows)
Done (151/151)

Processing 152/425: ZTF18abdiwpt... Saved 31 rows to ZTF18abdiwpt_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18abdiwpt_lightcurve.csv (31 rows)
Done (152/152)

Processing 153/425: ZTF18abdkimx... Saved 19 rows to ZTF18abdkimx_lightcurve.csv. Successfully c

...

Processing 260/425: ZTF18accrorf... Saved 22 rows to ZTF18accrorf_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18accrorf_lightcurve.csv (13 rows)
Done (260/260)

Processing 261/425: ZTF18accsawg... Saved 12 rows to ZTF18accsawg_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18accsawg_lightcurve.csv (8 rows)
Done (261/261)

Processing 262/425: ZTF18accwjiu... Saved 109 rows to ZTF18accwjiu_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18accwjiu_lightcurve.csv (57 rows)
Done (262/262)

Processing 263/425: ZTF18acdwuhy... Saved 55 rows to ZTF18acdwuhy_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18acdwuhy_lightcurve.csv (22 rows)
Done (263/263)

Processing 264/425: ZTF18acdyypo... Saved 104 rows to ZTF18acdyypo_lightcurve.csv. Successfully 

Processing 297/425: ZTF18acsohut... Saved 22 rows to ZTF18acsohut_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18acsohut_lightcurve.csv (8 rows)
Done (297/297)

Processing 298/425: ZTF18acspjqd... Saved 30 rows to ZTF18acspjqd_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18acspjqd_lightcurve.csv (11 rows)
Done (298/298)

Processing 299/425: ZTF18acsremz... Saved 137 rows to ZTF18acsremz_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18acsremz_lightcurve.csv (55 rows)
Done (299/299)

Processing 300/425: ZTF18acsxpmp... Saved 23 rows to ZTF18acsxpmp_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18acsxpmp_lightcurve.csv (11 rows)
Done (300/300)

Processing 301/425: ZTF18acsxusq... Saved 17 rows to ZTF18acsxusq_lightcurve.csv. Successfully c

Processing 334/425: ZTF18adaylyq... Saved 27 rows to ZTF18adaylyq_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18adaylyq_lightcurve.csv (13 rows)
Done (334/334)

Processing 335/425: ZTF18adazbol... Saved 24 rows to ZTF18adazbol_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18adazbol_lightcurve.csv (11 rows)
Done (335/335)

Processing 336/425: ZTF18adazgdh... Saved 52 rows to ZTF18adazgdh_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18adazgdh_lightcurve.csv (30 rows)
Done (336/336)

Processing 337/425: ZTF18adbadqp... Saved 13 rows to ZTF18adbadqp_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF18adbadqp_lightcurve.csv (8 rows)
Done (337/337)

Processing 338/425: ZTF18adbbhww... Saved 85 rows to ZTF18adbbhww_lightcurve.csv. Successfully cl

Processing 371/425: ZTF19aafnfyc... Saved 23 rows to ZTF19aafnfyc_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF19aafnfyc_lightcurve.csv (12 rows)
Done (371/371)

Processing 372/425: ZTF19aafwrhf... Saved 24 rows to ZTF19aafwrhf_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF19aafwrhf_lightcurve.csv (15 rows)
Done (372/372)

Processing 373/425: ZTF19aagmpeq... Saved 22 rows to ZTF19aagmpeq_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF19aagmpeq_lightcurve.csv (7 rows)
Done (373/373)

Processing 374/425: ZTF19aagmsrr... Saved 78 rows to ZTF19aagmsrr_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF19aagmsrr_lightcurve.csv (58 rows)
Done (374/374)

Processing 375/425: ZTF19aahjhoy... Saved 18 rows to ZTF19aahjhoy_lightcurve.csv. Successfully cl

Processing 408/425: ZTF19aalpoff... Saved 59 rows to ZTF19aalpoff_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF19aalpoff_lightcurve.csv (18 rows)
Done (408/408)

Processing 409/425: ZTF19aaluprf... Saved 25 rows to ZTF19aaluprf_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF19aaluprf_lightcurve.csv (13 rows)
Done (409/409)

Processing 410/425: ZTF19aalveag... Saved 35 rows to ZTF19aalveag_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF19aalveag_lightcurve.csv (19 rows)
Done (410/410)

Processing 411/425: ZTF19aalxcyo... Saved 19 rows to ZTF19aalxcyo_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/full_test_1/ZTF19aalxcyo_lightcurve.csv (11 rows)
Done (411/411)

Processing 412/425: ZTF19aalxlpp... Saved 22 rows to ZTF19aalxlpp_lightcurve.csv. Successfully c